# Langage SQL 

## BONUS : SQLAlchemy

In [1]:
import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect
from IPython.display import Markdown, display

1. SQLAlchemy¶
SQLAlchemy est une librairie Python très utilisée pour se connecter à un système de gestion de bases de données relationnelles, elle permet d'interpréter du code SQL.

On utilisera une base de données SQLite, plus légère et plus simple d'utilisation que la plupart des autres SGBDR.

Le premier objet important de ce module est l'engine. Il nous permet de nous connecter à une base de données et d'effectuer des requêtes SQL au format texte. La fonction create_engine crée une connexion avec une base de données. Si elle n'existe pas, la fonction va en créer une. L'argument est une chaîne de caractère qui a le format suivant : 'sqlite:///chemin_vers_la_base_de_donnees.db'

a) Créer une connexion à la base de données chinook.db que vous connaissez maintenant bien.

In [2]:
engine = create_engine('sqlite:///chinook.db', echo=True)

# En ajoutant l'argument echo = True, cela nous permet d'afficher la traduction en langage SQL des commandes du module


Une classe très pratique pour afficher des informations sur la base est l'Inspector, c'est un outil précieux pour comprendre à la fois le schéma des données ainsi que les contraintes qui s'opèrent entre elles.

b) Exécuter la cellule suivante afin d'instancier la classe Inspector dans une variable nommée inspector.

In [3]:
inspector = inspect(engine)


A l'aide de cet objet, on va pouvoir récupérer le nom des différentes tables qui constituent la base de données. La méthode get_table_names nous permet de le faire.

c) Exécuter la cellule suivante pour afficher les différentes tables présentes dans la base de données.

In [4]:
inspector.get_table_names()


2023-05-18 21:56:32,678 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-05-18 21:56:32,678 INFO sqlalchemy.engine.Engine [raw sql] ()


['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'media_types',
 'parcours',
 'playlist_track',
 'playlists',
 'sqlite_sequence',
 'sqlite_stat1',
 'tracks']

La méthode get_columns de la classe Inspector permet d'afficher les caractéristiques des différents attributs d'une table passée en argument.

d) Afficher les attributs de la table albums à l'aide de la cellule suivante.

In [5]:
inspector.get_columns(table_name='albums')


2023-05-18 21:56:47,910 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("albums")
2023-05-18 21:56:47,910 INFO sqlalchemy.engine.Engine [raw sql] ()


[{'name': 'AlbumId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'Title',
  'type': NVARCHAR(length=160),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'ArtistId',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

La méthode get_foreign_keys de l'objet inspector permet d'afficher les clés étrangères associées à une table passée en argument.

e) afficher les clés étrangères de la table albums à l'aide de la cellule suivante.

In [6]:
inspector.get_foreign_keys(table_name='albums')


2023-05-18 21:56:59,286 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("albums")
2023-05-18 21:56:59,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-18 21:56:59,289 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-05-18 21:56:59,290 INFO sqlalchemy.engine.Engine [raw sql] ('albums',)


[{'name': None,
  'constrained_columns': ['ArtistId'],
  'referred_schema': None,
  'referred_table': 'artists',
  'referred_columns': ['ArtistId'],
  'options': {}}]

Pour effectuer des requêtes SQL, on a besoin d'une connexion avec l'objet engine. Pour cela on utilise la méthode connect qui renvoie une instance de la classe Connection.

f) Instancier la classe Connection dans une variable nommée conn.

In [7]:
conn = engine.connect()


On peut désormais utiliser la méthode execute associée à une instance de la classe Connection créée précédemment.

Elle prend en argument une chaîne de caractères qui correspond à notre requête en langage SQL.

Pour obtenir le résultat de cette requête dans Python, on utilise la méthode fetchall associée à cet objet.

g) Afficher 10 lignes de la colonne Title de la table Albums.
Afin de s'assurer que notre requête soit bien interprétée par SQLAlchemy on utilisera la fonction text précédemment importée.

In [8]:
stmt = text("SELECT Title FROM albums LIMIT 10;")
result = conn.execute(stmt)
result.fetchall() 

2023-05-18 21:57:46,034 INFO sqlalchemy.engine.Engine SELECT Title FROM albums LIMIT 10;
2023-05-18 21:57:46,035 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()


[('For Those About To Rock We Salute You',),
 ('Balls to the Wall',),
 ('Restless and Wild',),
 ('Let There Be Rock',),
 ('Big Ones',),
 ('Jagged Little Pill',),
 ('Facelift',),
 ('Warner 25 Anos',),
 ('Plays Metallica By Four Cellos',),
 ('Audioslave',)]

h) Pour chaque commande de la table invoices, le prix est identifié par Total. Afficher le montant total des commandes par identifiant client CustomerId à l'aide de la méthode GROUP BY, on pourra les trier par ordre décroissant.

In [9]:
tot_price = text("SELECT invoices.CustomerId , SUM(Total) AS tot FROM invoices GROUP BY CustomerId ORDER BY tot DESC;")
result = conn.execute(tot_price)
result.fetchall() 

2023-05-18 21:57:58,291 INFO sqlalchemy.engine.Engine SELECT invoices.CustomerId , SUM(Total) AS tot FROM invoices GROUP BY CustomerId ORDER BY tot DESC;
2023-05-18 21:57:58,292 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ()


[(6, 49.620000000000005),
 (26, 47.620000000000005),
 (57, 46.62),
 (45, 45.62),
 (46, 45.62),
 (28, 43.620000000000005),
 (24, 43.62),
 (37, 43.62),
 (7, 42.62),
 (25, 42.62),
 (44, 41.620000000000005),
 (5, 40.620000000000005),
 (43, 40.620000000000005),
 (48, 40.62),
 (17, 39.620000000000005),
 (34, 39.620000000000005),
 (1, 39.62),
 (3, 39.62),
 (4, 39.62),
 (20, 39.62),
 (22, 39.62),
 (42, 39.62),
 (15, 38.620000000000005),
 (19, 38.620000000000005),
 (39, 38.620000000000005),
 (40, 38.620000000000005),
 (51, 38.620000000000005),
 (58, 38.620000000000005),
 (2, 37.620000000000005),
 (9, 37.620000000000005),
 (10, 37.620000000000005),
 (11, 37.620000000000005),
 (13, 37.620000000000005),
 (14, 37.620000000000005),
 (18, 37.620000000000005),
 (27, 37.620000000000005),
 (30, 37.620000000000005),
 (31, 37.620000000000005),
 (32, 37.620000000000005),
 (35, 37.620000000000005),
 (36, 37.620000000000005),
 (38, 37.620000000000005),
 (41, 37.620000000000005),
 (47, 37.620000000000005),
 (

Comme vous pouvez le voir, il est très simple d'exécuter des requêtes à l'aide de SQLAlchemy.

2. Création de table 
Nous allons reprendre le notebook numéro 4 donc vous allez à nouveau voir comment alimenter la base college.db avec les parcours pédagogiques d'un établissement d'enseignement et les étudiants qui suivent des cours là-bas, mais en utilisant cette fois-ci la syntaxe SQLAlchemy.

L'objet créé par la fonction MetaData permet de regrouper plusieurs informations concernant une ou plusieurs bases de données, notamment des informations sur les tables qui la composent, on utilisera cet objet lors de leurs créations.

i) Créer une base de données college.db.

In [10]:
engine = create_engine('sqlite:///college.db', echo=True)


j) Instancier un objet de la classe MetaData nommé meta.

In [11]:
meta = MetaData()

k) Créer une table parcours qui permettra d'identifier la filière de l'étudiant au sein de l'établissement. Cette table aura comme attributs l'identifiant id du parcours qui sera une clé primaire et son nom name.
l) Utiliser la méthode create_all associée à l'objet meta pour sauvegarder les changements sur la base de données (l'argument pris par cette fonction est un objet engine).

In [12]:
parcours = Table(
   'parcours', meta, 
   Column('id', Integer, primary_key=True), 
   Column('name', String)
)

meta.create_all(engine)


2023-05-18 21:58:47,813 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-18 21:58:47,814 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parcours")
2023-05-18 21:58:47,817 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-18 21:58:47,819 INFO sqlalchemy.engine.Engine COMMIT


Grâce au echo=True, on peut voir le code utilisé pour créer la table:

CREATE TABLE parcours (
    id INTEGER NOT NULL, 
    name VARCHAR, 
    PRIMARY KEY (id)
)
Qui n'est rien d'autre que la requête que vous aviez utilisée dans le précédent notebook pour créer la table parcours.

En SQL on choisit le nom de la table après le CREATE TABLE. Ensuite, entre parenthèses, on précise le nom des colonnes ainsi que leur type et des contraintes si jamais on veut en ajouter. Enfin on précise quelles sont les clés primaires.

m) créer une table students ayant comme attributs un identifiant id qui en sera la clé primaire, un prénom firstname au format String, un nom lastname au format String, ainsi que l'identifiant parcours_id du parcours suivi qui est une clé étrangère.
On pourra préciser une clé étrangère en utilisant la syntaxe suivante :

Column('colonne_name', type, ForeignKey("table_name.colonne_name"))

In [13]:
students = Table(
   'students', meta, 
   Column('id', Integer, primary_key=True), 
   Column('fistname', String),
   Column('lastname', String),
   Column('parcours_id', Integer, ForeignKey("parcours.id")) 
)

meta.create_all(engine)

2023-05-18 21:59:05,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-18 21:59:05,154 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parcours")
2023-05-18 21:59:05,155 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-18 21:59:05,157 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2023-05-18 21:59:05,158 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-18 21:59:05,159 INFO sqlalchemy.engine.Engine COMMIT


Remarquez comment SQL traduit cette nouvelle contrainte:

CREATE TABLE students (
    id INTEGER NOT NULL, 
    name VARCHAR, 
    lastname VARCHAR, 
    parcours_id INTEGER, 
    PRIMARY KEY (id), 
    FOREIGN KEY(parcours_id) REFERENCES parcours (id)

3. Modifications de table¶
Lorsque l'on modifie ou effectue une ou plusieurs opération(s) importante(s) sur une table, il est nécessaire de se prévenir de toute erreur de manipulation. Pour créer une transaction il faut connecter la machine à l'aide de la méthode connect de l'engine.

Une transaction a un début et une fin. Le début de la transaction est indiqué par la fonction begin associée à l'objet précédemment créé par la méthode connect.

Via cet objet, on pourra appeler la fonction rollback dans le cas où une opération de la transaction n'a pas été autorisée : on revient à l'état initial de la table.

La fonction commit permet elle de valider la transaction le cas échéant. En Python, cela nous donne la structure suivante:

# on crée la connection     
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'exécuter une transaction
        try:
            connection.execute(instruction)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()
L'argument instruction correspond à l'opération qui sera effectuée au sein de la transaction.

Lorsqu'on effectue une requête, qui est la forme de transaction la plus simple, ce fonctionnement est fait implicitement.

Insertion de lignes
Dans une table, on peut insérer ou supprimer des lignes. Ces opérations sont considérées comme des transactions et sont donc soumises aux règles qui assurent la cohérence de la base de données.

Pour insérer des tuples, on doit d'abord créer une liste qui contient des tuples. Ces tuples doivent représenter les valeurs des différents attributs de la table : la liste a donc la longueur du nombre de lignes qu'on veut insérer et chaque tuple a le même nombre d'entrées que la table.

n) Créer une liste values1 contenant les informations des 2 parcours de la table parcours, on les identifiera par (1, "Data Engineering") et (2, "Data Science"). Créer une liste values2 contenant les informations de 2 étudiants de la table students, on choisira les noms et prénoms que l'on souhaite. Attention cependant à bien respecter les contraintes liées à l'unicité de la clé primaire et à l'existence de la clé étrangère.

In [14]:
values1=[(1, "Data Engineering"), (2, "Data Science")]
values2=[(1, "Louise", "Mandon", 1), (2, "Baptiste", "Prost", 2)]


La syntaxe SQL pour ajouter un tuple dans une table est la suivante :

INSERT INTO nom_table 
VALUES tuple;
Cette commande est en langage SQL, et sera donc au format texte sous python si on souhaite l'exécuter.

On note 'INSERT INTO {tablename} VALUES ({markers})' où tablename sera le nom de la table et markers sera une variable explicitant le format des tuples que l'on injectera dans la table.

Une fois le tablename et le markers définis, on peut mettre à jour notre requête SQL en modifiant son format à l'aide de la méthode format de Python (doc) permettant des substitutions de valeurs et des mises en forme à l'intérieur de chaînes de caractères.

Cela permet que tablename et markers ne soient pas considérées comme une chaîne de caractères mais bien un argument.

   Pour définir correctement un markers, on peut utiliser la fonction suivante ','.join( '?' * len(values[0])) qui permet de définir un format correspondant à un tuple de la bonne longueur.
o) Exécuter la cellule suivante pour insérer dans la table parcours les valeurs précédemment définies dans la liste values1.

In [15]:
# on crée la connection
with engine.connect() as connection:
    # début de la transaction
    with connection.begin() as transaction:
        # on tente d'exécuter une transaction
        try:
            # On indique le format d'un tuple de cette table
            markers = ','.join('?' * len(values1[0])) 
            
            # On utilise le langage SQL en format texte où markers est le format d'un tuple
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            
            # On précise ce format particulier grâce à la fonction membre format
            ins = ins.format(tablename=parcours.name, markers=markers)
           
            # Enfin on peut utiliser les tuples créés en exécutant la commande SQL
            connection.execute(ins, values1)
        # si la transaction échoue
        except:
            transaction.rollback()
            raise
        # si la transaction réussit
        else:
            transaction.commit()

2023-05-18 21:59:39,650 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-18 21:59:39,651 INFO sqlalchemy.engine.Engine INSERT INTO parcours VALUES (?,?)
2023-05-18 21:59:39,651 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Data Engineering'), (2, 'Data Science')]
2023-05-18 21:59:39,653 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: parcours.id
[SQL: INSERT INTO parcours VALUES (?,?)]
[parameters: [(1, 'Data Engineering'), (2, 'Data Science')]]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

p) En utilisant une transaction SQLAlchemy, insérer dans la table students les valeurs précédemment définies dans la liste values2.


In [16]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values2[0])) 
            ins = 'INSERT INTO {tablename} VALUES ({markers})'
            
            ins = ins.format(tablename=students.name, markers=markers)
            connection.execute(ins, values2)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

2023-05-18 22:00:09,810 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-18 22:00:09,811 INFO sqlalchemy.engine.Engine INSERT INTO students VALUES (?,?,?,?)
2023-05-18 22:00:09,813 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Louise', 'Mandon', 1), (2, 'Baptiste', 'Prost', 2)]
2023-05-18 22:00:09,815 INFO sqlalchemy.engine.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: students.id
[SQL: INSERT INTO students VALUES (?,?,?,?)]
[parameters: [(1, 'Louise', 'Mandon', 1), (2, 'Baptiste', 'Prost', 2)]]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

q) Vous pouvez lancer la cellule suivante pour voir les éléments contenus dans les bases.


In [17]:
# Exécuter cette cellule
with engine.connect() as connection:
    results = connection.execute("SELECT * FROM students;")
    print(results.fetchall())

with engine.connect() as connection:
    results = connection.execute("SELECT * FROM parcours;")
    print(results.fetchall())

2023-05-18 22:00:27,585 INFO sqlalchemy.engine.Engine SELECT * FROM students;
2023-05-18 22:00:27,587 INFO sqlalchemy.engine.Engine [raw sql] ()
[(2, 'Marin', 'Dupont', 2)]
2023-05-18 22:00:27,589 INFO sqlalchemy.engine.Engine SELECT * FROM parcours;
2023-05-18 22:00:27,590 INFO sqlalchemy.engine.Engine [raw sql] ()
[(1, 'Data Engineering'), (2, 'Data Science')]


Suppression de table
Si l'on souhaite supprimer des lignes, il faut faire attention à la cohérence de la base de données. Dans notre exemple, on ne peut pas supprimer uniquement le tuple d'un des deux parcours de la table parcours. En effet, dans la table students un des attributs fait référence à la clé primaire d'un parcours. Cet attribut n'aurait donc plus de sens, et les transactions ACID nous permettent d'interdire ce genre de transaction.

   Si un parcours est retiré du programme de l'université et doit être retiré de la base, il faut aussi supprimer les lignes des étudiants associés (ici les étudiants suivent 1 seul cursus et sont donc malheureusement mis à la porte...) dans la table students. Pour supprimer ou modifier une ligne en particulier, il faut utiliser vos connaissances en langage SQL, notamment en ce qui concerne la recherche de tuple vérifiant une condition
On va procéder plus simplement ici et supprimer la table students.

r) Supprimer la table students de la base de donnée à l'aide des connaissances acquises jusqu'ici en SQL et SQLAlchemy. Puis vérifier qu'elle est bien supprimée en essayant d'y insérer un tuple.

In [18]:
sql = text('DROP TABLE IF EXISTS students;')
result = engine.execute(sql)


2023-05-18 22:00:43,360 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS students;
2023-05-18 22:00:43,361 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ()
2023-05-18 22:00:43,366 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
##Vérification que la table students n'existe plus en réutilisant le code d'insertion de tuple

values = [
    (1,"Jean", "Dubois", 1),
    (2,"Martin", "Dupont", 2),
    ]

with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            markers = ','.join('?' * len(values[0])) 
            ins = 'INSERT INTO students VALUES ({markers})'
            ins = ins.format(tablename=students.name, markers=markers)
            connection.execute(ins, values)
        except:
            transaction.rollback()
            raise
        else:
            transaction.commit()

2023-05-18 22:00:58,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-18 22:00:58,150 INFO sqlalchemy.engine.Engine INSERT INTO students VALUES (?,?,?,?)
2023-05-18 22:00:58,151 INFO sqlalchemy.engine.Engine [raw sql] [(1, 'Jean', 'Dubois', 1), (2, 'Martin', 'Dupont', 2)]
2023-05-18 22:00:58,153 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: students
[SQL: INSERT INTO students VALUES (?,?,?,?)]
[parameters: [(1, 'Jean', 'Dubois', 1), (2, 'Martin', 'Dupont', 2)]]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

Conclusion
Dans ce cours nous avons vu la structure à utiliser pour se connecter à une base de données et effectuer des requêtes sur une base de données relationnelles, ce qu'il faut retenir :

la structure de connexion à une base de données avec SQLAlchemy
la syntaxe pour effectuer des requêtes SQL à l'aide de SQLAlchemy
la syntaxe pour créer, modifier ou supprimer une table en SQL
N'hésitez pas à consulter notre Cheat Sheet sur SQL pour récapituler les notions étudiées lors de ce module.